## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-24-18-52-58 +0000,nypost,"Long Island kid stops burglar, leaps from wind...",https://nypost.com/2025/12/24/us-news/long-isl...
1,2025-12-24-18-51-39 +0000,nyt,Texas A&M Will Not Reinstate Lecturer Fired Ov...,https://www.nytimes.com/2025/12/24/us/texas-am...
2,2025-12-24-18-50-20 +0000,nypost,Long Island mayor facing Grinch accusations fo...,https://nypost.com/2025/12/24/us-news/long-isl...
3,2025-12-24-18-44-30 +0000,bbc,Explosion hits packed mosque in Nigeria,https://www.bbc.com/news/articles/c0je3v3en01o...
4,2025-12-24-18-44-00 +0000,wsj,Opinion | GOP ‘Moderate’ Hostage Taking,https://www.wsj.com/opinion/gop-moderate-hosta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2397/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,trump,24
24,christmas,21
35,new,19
175,epstein,15
41,national,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...,82
189,2025-12-24-01-08-43 +0000,bbc,"Trump trips, a fake video and 10 possible co-c...",https://www.bbc.com/news/articles/c5yj4yvx80do...,80
241,2025-12-23-20-42-27 +0000,latimes,"Supreme Court rules against Trump, bars Nation...",https://www.latimes.com/politics/story/2025-12...,77
179,2025-12-24-02-09-44 +0000,nypost,Trump administration to send 350 National Guar...,https://nypost.com/2025/12/23/us-news/trump-ad...,74
188,2025-12-24-01-15-18 +0000,bbc,US Supreme Court rejects Trump's bid to deplo...,https://www.bbc.com/news/articles/cy4xzn04x03o...,73


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,82,2025-12-24-10-30-00 +0000,wsj,Capital Account: President Trump’s social-medi...,https://www.wsj.com/economy/central-banking/wh...
42,64,2025-12-24-16-53-16 +0000,nypost,Afghan DC National Guard shooting suspect Rahm...,https://nypost.com/2025/12/24/us-news/afghan-d...
32,59,2025-12-24-17-22-59 +0000,nypost,DOJ seeks Christmas-week volunteers to speed u...,https://nypost.com/2025/12/24/us-news/doj-plea...
101,41,2025-12-24-12-26-16 +0000,nypost,Explosion in Moscow kills three people — inclu...,https://nypost.com/2025/12/24/world-news/explo...
54,40,2025-12-24-16-15-06 +0000,nyt,Zelensky Open to Pulling Back Troops in Easter...,https://www.nytimes.com/2025/12/24/world/europ...
78,37,2025-12-24-14-46-44 +0000,nypost,"Libya’s military chief, 7 others killed in pla...",https://nypost.com/2025/12/24/world-news/libya...
38,33,2025-12-24-17-08-39 +0000,nypost,Santa will spank you now: St. Nick turns up th...,https://nypost.com/2025/12/24/lifestyle/santas...
241,33,2025-12-23-20-42-27 +0000,latimes,"Supreme Court rules against Trump, bars Nation...",https://www.latimes.com/politics/story/2025-12...
34,32,2025-12-24-17-14-18 +0000,nypost,Repeat drunk driver accused of killing 18-year...,https://nypost.com/2025/12/24/us-news/la-drunk...
203,31,2025-12-23-23-07-22 +0000,latimes,L.A. woman's necklace was stolen and it held h...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
